---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-text-mining/resources/d9pwm) course resource._

---

# Assignment 1

In this assignment, you'll be working with messy medical data and using regex to extract relevant infromation from the data. 

Each line of the `dates.txt` file corresponds to a medical note. Each note has a date that needs to be extracted, but each date is encoded in one of many formats.

The goal of this assignment is to correctly identify all of the different date variants encoded in this dataset and to properly normalize and sort the dates. 

Here is a list of some of the variants you might encounter in this dataset:
* 04/20/2009; 04/20/09; 4/20/09; 4/3/09
* Mar-20-2009; Mar 20, 2009; March 20, 2009;  Mar. 20, 2009; Mar 20 2009;
* 20 Mar 2009; 20 March 2009; 20 Mar. 2009; 20 March, 2009
* Mar 20th, 2009; Mar 21st, 2009; Mar 22nd, 2009
* Feb 2009; Sep 2009; Oct 2010
* 6/2008; 12/2009
* 2009; 2010

Once you have extracted these date patterns from the text, the next step is to sort them in ascending chronological order accoring to the following rules:
* Assume all dates in xx/xx/xx format are mm/dd/yy
* Assume all dates where year is encoded in only two digits are years from the 1900's (e.g. 1/5/89 is January 5th, 1989)
* If the day is missing (e.g. 9/2009), assume it is the first day of the month (e.g. September 1, 2009).
* If the month is missing (e.g. 2010), assume it is the first of January of that year (e.g. January 1, 2010).
* Watch out for potential typos as this is a raw, real-life derived dataset.

With these rules in mind, find the correct date in each note and return a pandas Series in chronological order of the original Series' indices.

For example if the original series was this:

    0    1999
    1    2010
    2    1978
    3    2015
    4    1985

Your function should return this:

    0    2
    1    4
    2    0
    3    1
    4    3

Your score will be calculated using [Kendall's tau](https://en.wikipedia.org/wiki/Kendall_rank_correlation_coefficient), a correlation measure for ordinal data.

*This function should return a Series of length 500 and dtype int.*

In [26]:
import pandas as pd
import re
import numpy as np

doc = []
with open('dates.txt') as file:
    for line in file:
        doc.append(line)

df = pd.Series(doc)
df.head(10)

0         03/25/93 Total time of visit (in minutes):\n
1                       6/18/85 Primary Care Doctor:\n
2    sshe plans to move as of 7/8/71 In-Home Servic...
3                7 on 9/27/75 Audit C Score Current:\n
4    2/6/96 sleep studyPain Treatment Pain Level (N...
5                    .Per 7/06/79 Movement D/O note:\n
6    4, 5/18/78 Patient's thoughts about current su...
7    10/24/89 CPT Code: 90801 - Psychiatric Diagnos...
8                         3/7/86 SOS-10 Total Score:\n
9             (4/10/71)Score-1Audit C Score Current:\n
dtype: object

In [2]:
# #def date_sorter():
#         # Your code here
#     # Full date
# global df
# dates_extracted = df.str.extractall(r'(?P<origin>(?P<month>\d?\d)[/|-](?P<day>\d?\d)[/|-](?P<year>\d{4}))')
# index_left = ~df.index.isin([x[0] for x in dates_extracted.index])
# dates_extracted = dates_extracted.append(df[index_left].str.extractall(r'(?P<origin>(?P<month>\d?\d)[/|-](?P<day>([0-2]?[0-9])|([3][01]))[/|-](?P<year>\d{2}))'))
# index_left = ~df.index.isin([x[0] for x in dates_extracted.index])
# del dates_extracted[3]
# del dates_extracted[4]
# dates_extracted = dates_extracted.append(df[index_left].str.extractall(r'(?P<origin>(?P<day>\d?\d) ?(?P<month>[a-zA-Z]{3,})\.?,? (?P<year>\d{4}))'))
# index_left = ~df.index.isin([x[0] for x in dates_extracted.index])
# dates_extracted = dates_extracted.append(df[index_left].str.extractall(r'(?P<origin>(?P<month>[a-zA-Z]{3,})\.?-? ?(?P<day>\d\d?)(th|nd|st)?,?-? ?(?P<year>\d{4}))'))
# del dates_extracted[3]
# index_left = ~df.index.isin([x[0] for x in dates_extracted.index])

# # Without day
# dates_without_day = df[index_left].str.extractall('(?P<origin>(?P<month>[A-Z][a-z]{2,}),?\.? (?P<year>\d{4}))')
# dates_without_day = dates_without_day.append(df[index_left].str.extractall(r'(?P<origin>(?P<month>\d\d?)/(?P<year>\d{4}))'))
# dates_without_day['day'] = 1
# dates_extracted = dates_extracted.append(dates_without_day)
# index_left = ~df.index.isin([x[0] for x in dates_extracted.index])

# # Only year
# dates_only_year = df[index_left].str.extractall(r'(?P<origin>(?P<year>\d{4}))')
# dates_only_year['day'] = 1
# dates_only_year['month'] = 1
# dates_extracted = dates_extracted.append(dates_only_year)
# index_left = ~df.index.isin([x[0] for x in dates_extracted.index])

# # Year
# dates_extracted['year'] = dates_extracted['year'].apply(lambda x: '19' + x if len(x) == 2 else x)
# dates_extracted['year'] = dates_extracted['year'].apply(lambda x: str(x))
# # Month
# dates_extracted['month'] = dates_extracted['month'].apply(lambda x: x[1:] if type(x) is str and x.startswith('0') else x)
# month_dict = dict({'September': 9, 'Mar': 3, 'November': 11, 'Jul': 7, 'January': 1, 'December': 12,
#                    'Feb': 2, 'May': 5, 'Aug': 8, 'Jun': 6, 'Sep': 9, 'Oct': 10, 'June': 6, 'March': 3,
#                    'February': 2, 'Dec': 12, 'Apr': 4, 'Jan': 1, 'Janaury': 1,'August': 8, 'October': 10,
#                    'July': 7, 'Since': 1, 'Nov': 11, 'April': 4, 'Decemeber': 12, 'Age': 8})
# dates_extracted.replace({"month": month_dict}, inplace=True)
# dates_extracted['month'] = dates_extracted['month'].apply(lambda x: str(x))

# # Day
# dates_extracted['day'] = dates_extracted['day'].apply(lambda x: str(x))

# # Cleaned date
# dates_extracted['date'] = dates_extracted['month'] + '/' + dates_extracted['day'] + '/' + dates_extracted['year']
# dates_extracted['date'] = pd.to_datetime(dates_extracted['date'])

# dates_extracted.sort_values(by='date', inplace=True)
# df1 = pd.Series(list(dates_extracted.index.labels[0]))
    
# #    return df1# Your answer here
# dates_extracted = dates_extracted.reset_index().sort_values(by='level_0')
# dates_extracted['day'] = dates_extracted['day'].apply(int)
# dates_extracted['month'] = dates_extracted['month'].apply(int)
# dates_extracted['year'] = dates_extracted['year'].apply(int)
# answers = dates_extracted.apply(lambda x: str(x['year']) + "-" + str(x['month']) + "-" + str(x['day']), axis=1).tolist()

In [37]:
def get_date_regex():
    sep_l = r'[, \/\-\.]{1,2}'
    sep_d = r'[\/\- ]'

    month_l = r'([Jj]an\.?[a-z]*|[Ff]eb\.?[a-z]*|[Mm]ar\.?[a-z]*|[Aa]pr\.?[a-z]*|[Mm]ay\.?[a-z]*|[Jj]un\.?[a-z]*|[Jj]ul\.?[a-z]*|[Aa]ug\.?[a-z]*|[Ss]ep\.?(tember)?|[Oo]ct\.?[a-z]*|[Nn]ov\.?[a-z]*|[Dd]ec\.?[a-z]*)'
    month_d = r'(11|12|10|0?[123456789])'

    day = r'([0123]?\d)'
    year_l = f'y?((1|2)\d{{3}})'
    year = f"({year_l}|(\d{{2}}))"

    date = f"([^\w\-\.\/]|^|\()"\
    "("\
        "("\
            "("\
                f"(?P<mon_day>{month_d}{sep_d}{day}{sep_d})|"\
                f"(?P<day_mon>{day}{sep_d}{month_d}{sep_d})|"\
                f"(?P<MON_day>{month_l}{sep_l}{day}(th|nd|st)?{sep_l})|"\
                f"(?P<day_MON>{day}(th|nd|st)?{sep_l}{month_l}{sep_l})|"\
                f"(?P<MON>{month_l}{sep_l})|"\
                f"(?P<mon>{month_d}{sep_d})"\
                ")"\
            f"(?P<year>{year})"\
        ")|"\
        f"(?P<year_l>{year_l})"\
    ")"\
    f"([^\w\/]|$|[\),\.\;])"
    return date
def remove_beginning_error(text):
    month_l = r'(Jan\.?[a-z]*|Feb\.?[a-z]*|Mar\.?[a-z]*|Apr\.?[a-z]*|May\.?[a-z]*|Jun\.?[a-z]*|Jul\.?[a-z]*|Aug\.?[a-z]*|Sep\.?(tember)?|Oct\.?[a-z]*|Nov\.?[a-z]*|Dec\.?[a-z]*)'
    day = r'([0123]?\d)'
    m = re.match(f'^(?P<letter>[0-9a-zA-Z\.])({month_l})', text)
    if m is not None:
        letter = m.group('letter')
        text = re.sub(f"^{letter}", f"{letter} ", text)
        #print(f"^{letter}", f"{letter} ")
    m = re.match(f'^(?P<letter>[a-zA-Z\.])[0-3]\d', text)
    
    if m is not None:
        letter = m.group('letter')
        text = re.sub(f"^{letter}", f"{letter} ", text)
        #print(f"^{letter}", f"{letter} ")
    return text

def normalize_df(df):
    
    normalize_dict = {'-8/2000': ' 8/2000', 
                      '011/14/83':'11/14/83',
                      '.2010': ' 2010',
                      'n4/2004': ' 4/2004',
                      '.Feb 18, 1994': 'Feb 18, 1994',
                      '11, 2013': '11-2013',
                      
                      
                      'Age 16, 1991': 'Aug 16, 1991'
                      
                      
                     }
    def normalize_text(text):
        
        for key,value in normalize_dict.items():
            text = re.sub(key, value, text)
        text = remove_beginning_error(text)
        return text
    
    df = df.apply(normalize_text)
    
    return df

In [8]:
def parse_extract_date_parts(matched):

    month = None
    day = None
    year = None
    
    if matched['mon_day'] == matched['mon_day']:
        month = matched[5]
        day = matched[6]
    if matched['day_mon'] == matched['day_mon']:
        month = matched[9]
        day = matched[8]
    if matched['day_MON'] == matched['day_MON']:
        month = matched[18]
        day = matched[16]
    if matched['MON_day'] == matched['MON_day']:
        month = matched[11]
        day = matched[13]
    if matched['MON'] == matched['MON']:
        month = matched[21]
    if matched['mon'] == matched['mon']:
        month = matched[24]
    if matched['year'] == matched['year']:
        year = matched['year']
    if matched['year_l'] == matched['year_l']:
        year = matched[31]
    
    return {'month':month, 'day':day, 'year':year}

In [9]:
def month_letter2number(letter):
    if not isinstance(letter, str):
        return letter
    if len(letter) < 3:
        return int(letter)
    letter = letter.lower()
    letter_start = letter[0:3]
    letters = ['jan', 'feb', 'mar', 'apr', 'may', 'jun', 'jul', 'aug', 'sep', 'oct', 'nov', 'dec']
    numbers = list(range(1,13))
    let2num_dict = dict(zip(letters, numbers))
    
    if letter_start in letters:
        return let2num_dict[letter[0:3]]
    
    raise Exception(f"{letter} could not be converted to month number")
    return 

In [40]:
def date_sorter():
    global df
    df = normalize_df(df)
    date = get_date_regex()
    matched = df.str.extractall(date, flags=re.IGNORECASE)
    matched = matched.reset_index()

    month_day_year = matched.apply(lambda x: parse_extract_date_parts(x), axis=1)
    matched[['day','month','year']] =pd.DataFrame.from_records(month_day_year)
    matched['month'] = matched['month'].apply(month_letter2number)

    matched['cnt'] = matched.apply(lambda x: sum([item is not None for item in [x['month'] , x['day'], x['year']]]), axis=1)

    date_match =matched.groupby('level_0').apply(lambda x: x.sort_values(by=['cnt', 'year'], ascending=False).head(1))[['day', 'month', 'year']]

    date_match['day'] = date_match['day'].apply(lambda x: int(x) if x is not None  else 1)
    date_match['month'] = date_match['month'].apply(lambda x: int(x) if x is not None and x == x else 1)
    date_match['year'] = date_match['year'].apply(lambda x: int(x) + 1900 if int(x) < 100 else int(x))


    index = date_match.sort_values(by=['year', 'month','day']).reset_index()['level_0']

    return index


In [43]:
# my_answer = date_match.apply(lambda x: str(x['year']) + "-" + str(x['month']) + "-" + str(x['day']), axis=1).tolist()

# same = [answers[id] == my_answer[id] for id in list(range(0,500))]

# ids = pd.DataFrame({"same":same, 'id':list(range(0,500))}).query('not same')['id'].tolist()

In [44]:
# len(ids)

# [(my_answer[id], answers[id], df[id]) for id in ids]

In [45]:
# [text for text in df if 'Age' in text ]

In [ ]:
#dates = date_match.apply(lambda x: f"{x['month']}-{x['day']}-{x['year']}", axis=1).tolist()

#list(zip(dates, df))

In [ ]:
# [df[id] for id in list(set(list(range(0,499))).difference(set(index.tolist()) ))]

In [ ]:
#date_sorter(df)
# len(index)

In [ ]:
# month_ids = matched.query('month!=month')['level_0'].tolist()
# day_ids = [ id for id in matched.query('day!=day')['level_0'].tolist() if id not in month_ids]
# [df[id] for id in day_ids]

# month_ids

# matched.query('level_0 == 80').sort_values(by=['cnt', 'year'], ascending=False).head(1)['']